### Tracking the two pedulums

In [ ]:
import cv2
import csv
import os
import numpy as np

# Global variables for custom ROI selection
circle_center = None
circle_radius = 0
drawing = False

def select_circle(event, x, y, flags, param):
    """Mouse callback function for selecting a circular ROI."""
    global circle_center, circle_radius, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        # Start drawing
        circle_center = (x, y)
        circle_radius = 0
        drawing = True

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        # Update radius as the mouse moves
        circle_radius = int(np.sqrt((x - circle_center[0])**2 + (y - circle_center[1])**2))

    elif event == cv2.EVENT_LBUTTONUP:
        # Finish drawing
        drawing = False


def custom_roi_selection(frame, title="Select Object"):
    """Custom circular ROI selection."""
    global circle_center, circle_radius, drawing

    clone = frame.copy()
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, select_circle)

    while True:
        temp_frame = clone.copy()
        if circle_center and circle_radius > 0:
            # Draw the circle as the user selects
            cv2.circle(temp_frame, circle_center, circle_radius, (0, 255, 0), 2)
        cv2.imshow(title, temp_frame)

        # Break on Enter key
        key = cv2.waitKey(1) & 0xFF
        if key == 13:  # Enter key
            break
        elif key == 27:  # Escape key
            circle_center, circle_radius = None, 0
            break

    cv2.destroyWindow(title)
    if circle_center and circle_radius > 0:
        # Convert circle to bounding box
        x = circle_center[0] - circle_radius
        y = circle_center[1] - circle_radius
        w = circle_radius * 2
        h = circle_radius * 2
        return (x, y, w, h)
    else:
        return None


def process_video(video_path, output_csv):
    """Process a single video."""
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Error: Could not open video {video_path}.")
        return

    # Read the first frame
    success, frame = video.read()
    if not success:
        print(f"Error: Could not read video {video_path}.")
        return

    # Select ROIs manually using the custom selector
    print("Select Object 1:")
    bbox1 = custom_roi_selection(frame, "Select Object 1")
    if not bbox1:
        print("Selection canceled for Object 1.")
        return

    print("Select Object 2:")
    bbox2 = custom_roi_selection(frame, "Select Object 2")
    if not bbox2:
        print("Selection canceled for Object 2.")
        return

    # Initialize trackers for the selected objects
    tracker1 = cv2.TrackerCSRT_create()
    tracker2 = cv2.TrackerCSRT_create()
    tracker1.init(frame, bbox1)
    tracker2.init(frame, bbox2)

    coords_obj1 = []
    coords_obj2 = []
    frame_count = 0

    while True:
        success, frame = video.read()
        if not success:
            print(f"End of video {video_path}.")
            break

        # Process every 10th frame only
        if frame_count % 10 == 0:
            # Update trackers and store coordinates
            success1, bbox1 = tracker1.update(frame)
            success2, bbox2 = tracker2.update(frame)

            if success1:
                coords_obj1.append((bbox1[0], bbox1[1]))
                (x1, y1, w1, h1) = [int(v) for v in bbox1]
                cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)
                cv2.putText(frame, "Object 1", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            else:
                coords_obj1.append((None, None))

            if success2:
                coords_obj2.append((bbox2[0], bbox2[1]))
                (x2, y2, w2, h2) = [int(v) for v in bbox2]
                cv2.rectangle(frame, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 0), 2)
                cv2.putText(frame, "Object 2", (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                coords_obj2.append((None, None))

            # Display the frame
            cv2.imshow("Tracking", frame)

        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

    # Save coordinates to CSV
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Frame", "Object 1 X", "Object 1 Y", "Object 2 X", "Object 2 Y"])
        for i, (coord1, coord2) in enumerate(zip(coords_obj1, coords_obj2)):
            writer.writerow([i * 10, coord1[0], coord1[1], coord2[0], coord2[1]])

    print(f"Coordinates saved to {output_csv}")


def process_directory(input_folder, output_folder):
    """Process videos in a folder."""
    video_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

    if not video_files:
        print("No video files found.")
        return

    print("Available video files:")
    for idx, file in enumerate(video_files):
        print(f"{idx}: {file}")

    file_index = int(input("Enter the index of the file to process: "))
    if file_index < 0 or file_index >= len(video_files):
        print("Invalid index.")
        return

    video_path = os.path.join(input_folder, video_files[file_index])
    output_csv = os.path.join(output_folder, f"{os.path.splitext(video_files[file_index])[0]}_coordinates.csv")
    os.makedirs(output_folder, exist_ok=True)

    print(f"Processing video: {video_path}")
    process_video(video_path, output_csv)


# Main Program
input_folder = R"C:\Users\avsha\Documents\vid_short"
output_folder = R"C:\Users\avsha\Documents\vid_out_csv"
process_directory(input_folder, output_folder)


Available video files:
0: compressed_S6130001.MP4
1: compressed_S6130002.MP4
2: compressed_S6130003.MP4
3: compressed_S6130005.MP4
4: compressed_S6130006.MP4
5: compressed_S6130007.MP4
6: compressed_S6130008.MP4
7: compressed_S6130009.MP4
8: compressed_S6130010.MP4
9: compressed_S6160001.MP4
10: compressed_S6160002.MP4
11: compressed_S6160003.MP4
12: compressed_S6160004.MP4
13: compressed_S6160005.MP4
14: compressed_S6160006.MP4
Processing video: C:\Users\avsha\Documents\vid_short\compressed_S6130003.MP4
Select Object 1:


KeyboardInterrupt: 

In [ ]:
import cv2
import os
import csv
import numpy as np

def detect_pendulums(frame):
    """Detect two pendulums in the frame and return their bounding boxes."""
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (15, 15), 0)

    # Threshold the image
    _, thresh = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours by size and sort by x-coordinate
    pendulums = sorted([cv2.boundingRect(c) for c in contours if cv2.contourArea(c) > 500], key=lambda x: x[0])

    if len(pendulums) >= 2:
        return pendulums[:2]
    else:
        raise ValueError("Could not detect two pendulums in the frame.")

def process_video(video_path, output_csv):
    """Process a single video and save tracking results to CSV."""
    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print(f"Error: Could not open video {video_path}.")
        return

    # Read the first frame
    success, frame = video.read()
    if not success:
        print(f"Error: Could not read video {video_path}.")
        return

    # Auto-detect pendulums
    try:
        pendulums = detect_pendulums(frame)
        bbox1, bbox2 = pendulums[0], pendulums[1]
    except ValueError as e:
        print(f"Error: {e} for video {video_path}")
        return

    # Initialize trackers
    tracker1 = cv2.TrackerCSRT_create()
    tracker2 = cv2.TrackerCSRT_create()
    tracker1.init(frame, bbox1)
    tracker2.init(frame, bbox2)

    coords_obj1 = []
    coords_obj2 = []
    frame_count = 0
    processed_frame_count = 0

    while True:
        success, frame = video.read()
        if not success:
            print(f"End of video {video_path}.")
            break

        if frame_count % 10 == 0:
            processed_frame_count += 1

            # Update trackers
            success1, bbox1 = tracker1.update(frame)
            success2, bbox2 = tracker2.update(frame)

            if success1:
                (x1, y1, w1, h1) = [int(v) for v in bbox1]
                coords_obj1.append((x1, y1))
                cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)
                cv2.putText(frame, "Object 1", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            else:
                coords_obj1.append((None, None))
                cv2.putText(frame, "Object 1 Lost", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

            if success2:
                (x2, y2, w2, h2) = [int(v) for v in bbox2]
                coords_obj2.append((x2, y2))
                cv2.rectangle(frame, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 0), 2)
                cv2.putText(frame, "Object 2", (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                coords_obj2.append((None, None))
                cv2.putText(frame, "Object 2 Lost", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

            # Display frame
            cv2.imshow("Two-Object Tracking", frame)

        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

    # Save coordinates to CSV
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Frame", "Object 1 X", "Object 1 Y", "Object 2 X", "Object 2 Y"])
        for i, (coord1, coord2) in enumerate(zip(coords_obj1, coords_obj2)):
            writer.writerow([i * 10, coord1[0], coord1[1], coord2[0], coord2[1]])

    print(f"Tracking results saved to {output_csv}")

def process_directory(directory_path, output_directory):
    """Process all videos in a directory."""
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # List video files with case-insensitive extension matching
    video_files = [f for f in os.listdir(directory_path) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    if not video_files:
        print("No video files found in the directory.")
        return

    print(f"Found {len(video_files)} video(s) in the directory.")

    for video_file in video_files:
        video_path = os.path.join(directory_path, video_file)
        print(f"Processing {video_path}...")
        output_csv = os.path.join(output_directory, f"{os.path.splitext(video_file)[0]}_coordinates.csv")
        process_video(video_path, output_csv)


# Main directory processing
input_directory = R"C:\Users\avsha\Documents\vid_short"
output_directory = R"C:\Users\avsha\Documents\vid_out_csv"
process_directory(input_directory, output_directory)


Found 15 video(s) in the directory.
Processing C:\Users\avsha\Documents\vid_short\compressed_S6130001.MP4...
Tracking results saved to C:\Users\avsha\Documents\vid_out_csv\compressed_S6130001_coordinates.csv
Processing C:\Users\avsha\Documents\vid_short\compressed_S6130002.MP4...
Error: Could not detect two pendulums in the frame. for video C:\Users\avsha\Documents\vid_short\compressed_S6130002.MP4
Processing C:\Users\avsha\Documents\vid_short\compressed_S6130003.MP4...
Tracking results saved to C:\Users\avsha\Documents\vid_out_csv\compressed_S6130003_coordinates.csv
Processing C:\Users\avsha\Documents\vid_short\compressed_S6130005.MP4...
Tracking results saved to C:\Users\avsha\Documents\vid_out_csv\compressed_S6130005_coordinates.csv
Processing C:\Users\avsha\Documents\vid_short\compressed_S6130006.MP4...


KeyboardInterrupt: 